<h1 align="center"> Recording Measurement Error Mitigation </h1>

In [1]:
# Installing Libraries
!pip install qiskit pylatexenc --quiet
!pip install qiskit.ignis --quiet

In [6]:
# Importing Libraries
from qiskit import *
from qiskit.tools.monitor import  job_monitor
from qiskit.tools.visualization import plot_state_qsphere
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex

import numpy as np
from qiskit.extensions import Initialize
from qiskit.ignis.verification import marginal_counts
from qiskit.quantum_info import random_statevector

# For Error
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error

import warnings
warnings.filterwarnings('ignore')

## Noise Model Function

In [7]:
def get_noise(p):
    error_measure = pauli_error([("X",p), ("I", 1-p)])
    noise_model = NoiseModel()
    
    # apply measurement error to noise model
    noise_model.add_all_qubit_quantum_error(error_measure, "measure")
    
    return noise_model

## Create Noise Model

In [8]:
noise_model = get_noise(0.01)  # 1% chance of error

## Effects of Noise Model

### Without noise Model

In [10]:

all_state = ['00', '01', '10', '11']

for state in all_state:
    qc = QuantumCircuit(2,2)
    
    if state[0] == '1':
        qc.x(1)
    if state[1] == '1':
        qc.x(0)
    
    qc.measure([0,1],[0,1])
    
    # Run on Simulator
    simulator = Aer.get_backend('aer_simulator')     # Select Simulator
    t_tc = transpile(qc, simulator)
    qobj = assemble(t_tc, shots =10000)                    
    counts = simulator.run(qobj, noise_model=None).result().get_counts()
    print(state + ' becomes ', counts)

00 becomes  {'00': 10000}
01 becomes  {'01': 10000}
10 becomes  {'10': 10000}
11 becomes  {'11': 10000}


### Without Noise Model

In [12]:
all_state = ['00', '01', '10', '11']

for state in all_state:
    qc = QuantumCircuit(2,2)
    
    if state[0] == '1':
        qc.x(1)
    if state[1] == '1':
        qc.x(0)
    
    qc.measure([0,1],[0,1])
    
    # Run on Simulator
    simulator = Aer.get_backend('aer_simulator')     # Select Simulator
    t_tc = transpile(qc, simulator)
    qobj = assemble(t_tc, shots =10000)                    
    counts = simulator.run(qobj, noise_model=noise_model).result().get_counts()
    print(state + ' becomes ', counts)

00 becomes  {'00': 9779, '01': 113, '10': 106, '11': 2}
01 becomes  {'01': 9789, '00': 107, '11': 103, '10': 1}
10 becomes  {'10': 9792, '11': 105, '00': 103}
11 becomes  {'11': 9794, '01': 102, '10': 103, '00': 1}
